In [6]:
import numpy as np
import os
import os.path as op
import h5py
import shutil
import requests
import statistics

In [7]:
outpath = '/Users/aleex/DSE/Machine Learning/Project/label/'

In [11]:
def createData(outpath):
    # PATH TO STORE THE TRAINING SAMPLES
    #outpath = '/Users/aleex/DSE/Machine Learning/Project/label/'
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    def genLabel(goes16, goes17):
        n1 = len(goes16)*len(goes16[0])
        n2 = len(goes17)*len(goes17[0])

        std1 = np.std(goes16)
        std2 = np.std(goes17)

        diff = goes16 - goes17
        diff_std = np.std(diff)

        delta = abs(goes16 - goes17)
        label = delta//diff_std
        label = label.astype(int)

        return label

    def writeH5(data, label, uniTime):
        coords = np.array(data).astype(np.float)
        shape = coords.shape
        hdf5_path = outpath + uniTime + '.h5'
        with h5py.File(hdf5_path, mode='w') as f:
            d = f.create_dataset('/data', data = coords)
            l = f.create_dataset('/label', data = label)

    #ROOT URL OF THE NPY FILES
    skip = False
    rootURL = "https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/"
    for idx in range(98, 109):    
        for h in range(0, 24):
            for m in range(0, 6):
                data = np.empty((401,1001,0), float)
                label = np.empty((401,1001,0), int)
                for band in range(7, 11):
                    with open('log2', 'a') as log:
                        timeID = "s2019{}{}{}".format(str(idx).zfill(3), str(h).zfill(2), str(m*10).zfill(2))

                        goes16_url = rootURL + "{}/OR_ABI-L1b-RadF-M6C{}_G16_{}".format(str(idx).zfill(3), str(band).zfill(2), timeID)
                        goes17_url = rootURL + "{}/OR_ABI-L1b-RadF-M6C{}_G17_{}".format(str(idx).zfill(3), str(band).zfill(2), timeID)
                        
                        
                        try:
                            response16 = requests.get(goes16_url, stream=True)
                            response17 = requests.get(goes17_url, stream=True)
                            
                            with open('temp16.npy', 'wb') as temp16:
                                shutil.copyfileobj(response16.raw, temp16)
                            with open('temp17.npy', 'wb') as temp17:
                                shutil.copyfileobj(response17.raw, temp17)               

                            data16 = np.load('temp16.npy').reshape((401, 1001, 1))
                            data17 = np.load('temp17.npy').reshape((401, 1001, 1))
                            
                            labels = genLabel(data16, data17)
                            
                            data = np.append(data, data17, axis = 2)
                            #print(data)
                            
                            label = np.append(label, labels, axis = 2)
                            
                            #vote = np.round(np.median(label, axis=2))
                            #writeH5(stack, vote, timeID)

                        except Exception as e:
                            skip = True
                            print("url doesn't exist for " + str(goes16_url))
                            print(e)
                            print('\n')
                            print(str(goes16_url), file=log)
                            print('\n')
                            print(e, file=log)
                            break
                if skip:
                    skip=False
                    continue
                
                data = data.reshape((401*1001, -1))
                vote = np.round(np.median(label, axis=2)).astype('int')                
                vote = vote.reshape((-1))
                writeH5(data, vote, timeID)

In [12]:
createData(outpath)

url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970000
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970010
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970020
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970030
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970040
Cannot load file containing pickled data when allow_pickle=False




url doesn'

url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970700
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970710
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970720
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970730
Cannot load file containing pickled data when allow_pickle=False




url doesn't exist for https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/097/OR_ABI-L1b-RadF-M6C07_G16_s20190970740
Cannot load file containing pickled data when allow_pickle=False




url doesn'

KeyboardInterrupt: 